In [15]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%matplotlib widget

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import xarray as xr
import pandas as pd

import pycomlink as pycml

In [18]:
cmls = xr.open_dataset('../pycomlink/io/example_data/example_cml_data.nc')
cml_list = [cmls.isel(cml_id=i) for i in range(len(cmls.cml_id))]

for cml in cml_list:
    cml['tsl'] = cml.tsl.where(cml.tsl != 255.0)
    cml['rsl'] = cml.rsl.where(cml.rsl != -99.9)
    cml['trsl'] = cml.tsl - cml.rsl

# Test new xarray_wrapper decorator that checks for a `time` dimension loops over remaining dims 

In [19]:
# some tests for using ufunc

# calc manually for one channel
def baseline_constant(trsl, wet, n_average_last_dry):
    print(trsl.shape)
    baseline = np.zeros_like(trsl, dtype=np.float64)
    baseline[0:n_average_last_dry] = trsl[0:n_average_last_dry]
    for i in range(n_average_last_dry, len(trsl)):
        if np.isnan(wet[i]):
            baseline[i] = np.NaN
        elif wet[i] & ~wet[i-1]:
            baseline[i] = np.mean(baseline[(i-n_average_last_dry) : i])
        elif wet[i] & wet[i-1]:
            baseline[i] = baseline[i - 1]
        else:
            baseline[i] = trsl[i]
    return baseline

baseline = baseline_constant(
    cml.trsl.isel(channel_id=0).values, 
    cml.trsl.rolling(time=60, center=True).std().isel(channel_id=0).values > 0.3, 
    n_average_last_dry=10,
)

plt.figure()
plt.plot(cml.trsl.isel(channel_id=0).values)
plt.plot(baseline);

(15840,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# use ufunc 
plt.figure()

cml.trsl.isel(channel_id=0).plot.line(x='time')
xr.apply_ufunc(
    baseline_constant, 
    cml.trsl, 
    cml.trsl.rolling(time=60, center=True).std().isel(channel_id=0) > 0.3, 
    10, 
    input_core_dims=[['time'], ['time'], []],
    output_core_dims=[['time']],
    vectorize=True,
).isel(channel_id=0).plot.line(x='time');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(15840,)
(15840,)


In [21]:
# build a decorator with the ufunc solution, here only for
# baseline_constant as above, not handling different number of args
from functools import wraps

def decorator(func):
    @wraps(func)
    def inner(*args, **kwargs):
        
        return xr.apply_ufunc(
            func,
            *args,
            input_core_dims=[['time'], ['time'], []],
            output_core_dims=[['time']],
            vectorize=True,
        )

    return inner

f = decorator(baseline_constant)
cml['baseline'] = f(cml.trsl, cml.trsl.rolling(time=60, center=True).std().isel(channel_id=0) > 0.3, 10)
cml.baseline

(15840,)
(15840,)


<xarray.DataArray 'baseline' (channel_id: 2, time: 15840)>
array([[58.        , 58.        , 58.        , ..., 58.        ,
        58.        , 58.        ],
       [57.29999924, 57.        , 57.        , ..., 57.        ,
        57.        , 57.        ]])
Coordinates:
  * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
    cml_id            <U17 'SY8534_2_SY2370_2'
    length            float64 4.994
    site_a_latitude   float64 57.07
    site_a_longitude  float64 2.09
    site_b_latitude   float64 57.07
    site_b_longitude  float64 2.023
  * channel_id        (channel_id) object 'channel_1' 'channel_2'
    frequency         (channel_id) float64 ...
    polarization      (channel_id) object ...

In [183]:
# Function required in new wrapper
import inspect

def _get_new_args_dict(func, args, kwargs):
    from collections import OrderedDict
    new_args_dict = OrderedDict()
    for i, (arg, parameter) in enumerate(inspect.signature(func).parameters.items()):
        if i < len(args):
            new_args_dict[arg] = args[i]
        elif arg in kwargs.keys():
            new_args_dict[arg] = kwargs[arg]
        else:
            new_args_dict[arg] = parameter.default

    return new_args_dict

func = calc_R_from_A
args = [cml.trsl, cml.length, cml.frequency]
kwargs = {'R_min': 0.815}
_get_new_args_dict(func, args, kwargs)

OrderedDict([('A',
              <xarray.DataArray 'trsl' (channel_id: 2, time: 15840)>
              array([[58. , 58. , 58. , ..., 58. , 58. , 58. ],
                     [57.3, 57. , 57. , ..., 57. , 57. , 57. ]], dtype=float32)
              Coordinates:
                * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
                  cml_id            <U17 'SY8534_2_SY2370_2'
                  length            float64 4.994
                  site_a_latitude   float64 57.07
                  site_a_longitude  float64 2.09
                  site_b_latitude   float64 57.07
                  site_b_longitude  float64 2.023
                * channel_id        (channel_id) object 'channel_1' 'channel_2'
                  frequency         (channel_id) float64 2.497e+10 2.598e+10
                  polarization      (channel_id) object 'V' 'V'),
             ('L_km',
              <xarray.DataArray 'length' ()>
              array(4.994139)
              Coor

In [184]:
# build a decorator with the ufunc solution, now dynamically
# adjusting to different args

from functools import wraps

def xarray_apply_along_time_dim(input_core_dims):

    def decorator(func):
        @wraps(func)
        def inner(*args, **kwargs):
            new_args_dict = _get_new_args_dict(func=func, args=args, kwargs=kwargs)
            print(new_args_dict)
            input_core_dims = []
            found_time_dim = False
            for arg in new_args_dict.values():
                print('arg')
                try:
                    if 'time' in arg.dims:
                        found_time_dim = True
                        input_core_dims.append(['time'])
                    else:
                        input_core_dims.append([])
                except AttributeError:
                    input_core_dims.append([])
             
            if not found_time_dim:
                return func(*args, **kwargs)
            else:
                print(list(input_core_dims))
                return xr.apply_ufunc(
                    func,
                    *list(new_args_dict.values()),
                    input_core_dims=input_core_dims,
                    output_core_dims=[['time']],
                    vectorize=True,
                )

        return inner
    
    return decorator

decorator = xarray_apply_along_time_dim(input_core_dims=[['time'], ['time'], []])
f = decorator(baseline_constant)
cml['baseline'] = f(cml.trsl, cml.trsl.rolling(time=60, center=True).std().isel(channel_id=0) > 0.3, 10)

OrderedDict([('trsl', <xarray.DataArray 'trsl' (channel_id: 2, time: 15840)>
array([[58. , 58. , 58. , ..., 58. , 58. , 58. ],
       [57.3, 57. , 57. , ..., 57. , 57. , 57. ]], dtype=float32)
Coordinates:
  * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
    cml_id            <U17 'SY8534_2_SY2370_2'
    length            float64 4.994
    site_a_latitude   float64 57.07
    site_a_longitude  float64 2.09
    site_b_latitude   float64 57.07
    site_b_longitude  float64 2.023
  * channel_id        (channel_id) object 'channel_1' 'channel_2'
    frequency         (channel_id) float64 2.497e+10 2.598e+10
    polarization      (channel_id) object 'V' 'V'), ('wet', <xarray.DataArray 'trsl' (time: 15840)>
array([False, False, False, ..., False, False, False])
Coordinates:
  * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
    cml_id            <U17 'SY8534_2_SY2370_2'
    length            float64 4.994
    site_a_latitude   float64 

In [182]:
#@xarray_apply_along_time_dim(input_core_dims=[['time'], [], [], [], [], [], []])
@xarray_apply_along_time_dim(input_core_dims=[])
def calc_R_from_A(A, L_km, f_GHz=None, a=None, b=None, pol="H", R_min=0.1):

    if f_GHz is not None:
        a, b = pycml.processing.k_R_relation.a_b(f_GHz, pol=pol)
    
    R = np.zeros_like(A)

    nan_index = np.isnan(A)
    R[nan_index] = np.nan

    # This ignores the numpy warning stemming from A >=0 where A contains NaNs
    with np.errstate(invalid="ignore"):
        R[~nan_index & (A >= 0)] = (A[~nan_index & (A >= 0)] / (a * L_km)) ** (1 / b)
        R[~nan_index & (R < R_min)] = 0
    return R

calc_R_from_A(cml.trsl, cml.length, cml.frequency/1e9)

OrderedDict([('A', <xarray.DataArray 'trsl' (channel_id: 2, time: 15840)>
array([[58. , 58. , 58. , ..., 58. , 58. , 58. ],
       [57.3, 57. , 57. , ..., 57. , 57. , 57. ]], dtype=float32)
Coordinates:
  * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
    cml_id            <U17 'SY8534_2_SY2370_2'
    length            float64 4.994
    site_a_latitude   float64 57.07
    site_a_longitude  float64 2.09
    site_b_latitude   float64 57.07
    site_b_longitude  float64 2.023
  * channel_id        (channel_id) object 'channel_1' 'channel_2'
    frequency         (channel_id) float64 2.497e+10 2.598e+10
    polarization      (channel_id) object 'V' 'V'), ('L_km', <xarray.DataArray 'length' ()>
array(4.994139)
Coordinates:
    cml_id            <U17 'SY8534_2_SY2370_2'
    length            float64 4.994
    site_a_latitude   float64 57.07
    site_a_longitude  float64 2.09
    site_b_latitude   float64 57.07
    site_b_longitude  float64 2.023), ('f_GHz', <xar

<xarray.DataArray (channel_id: 2, time: 15840)>
array([[72.26416 , 72.26416 , 72.26416 , ..., 72.26416 , 72.26416 ,
        72.26416 ],
       [67.744286, 67.40757 , 67.40757 , ..., 67.40757 , 67.40757 ,
        67.40757 ]], dtype=float32)
Coordinates:
  * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
    cml_id            <U17 'SY8534_2_SY2370_2'
    length            float64 4.994
    site_a_latitude   float64 57.07
    site_a_longitude  float64 2.09
    site_b_latitude   float64 57.07
    site_b_longitude  float64 2.023
  * channel_id        (channel_id) object 'channel_1' 'channel_2'
    frequency         (channel_id) float64 2.497e+10 2.598e+10
    polarization      (channel_id) object 'V' 'V'

In [31]:
# not finished solution that is meant to first build a dict with info from args and kwargs for each dim

from functools import wraps
from collections import defaultdict

def decorator(func):
    @wraps(func)
    def inner(*args, **kwargs):
        
        dims_dict = defaultdict(dict)
        for i, arg in enumerate(args):
            try:
                dims = arg.dims
                for dim in dims:
                    print(dim)
                    dims_dict[dim]: {'type': 'arg', 'key': i, 'dim_len': len(arg[dim])}
                    print(dims_dict)
            except AttributeError:
                pass
        
#         if len(dims_to_loop_over) > 1:
#             raise NotImplementedError('Looping over more than one dim is not yet implemented.')
            
        data_list = []
        
        # iterate over the other dims
            
        print(dims_dict)
    return inner

test_func = decorator(pycml.processing.baseline.baseline_constant)
#test_func(trsl=cml.rsl, wet=cml.rsl > 0);
test_func(cml.rsl, cml.rsl > 0);

channel_id
defaultdict(<class 'dict'>, {})
time
defaultdict(<class 'dict'>, {})
channel_id
defaultdict(<class 'dict'>, {})
time
defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})


In [22]:
# old version that would have been too complicated because it currently
# is not possible to know how often to iterate over the non-time dim

from functools import wraps

def decorator(func):
    @wraps(func)
    def inner(*args, **kwargs):
        
        # Find args and kwargs with time dim
        args_with_xarray_time_dim = []
        kwargs_with_xarray_time_dim = []
        for i, arg in enumerate(args):
            try:
                if 'time' in arg.dims:
                    args_with_xarray_time_dim.append({i: arg})
            except AttributeError:
                pass
        for kwarg_key, kwarg_value in kwargs.items():
            try:
                if 'time' in kwarg_value.dims:
                    kwargs_with_xarray_time_dim.append({kwarg_key: kwarg_value})
            except AttributeError:
                pass

        print(args_with_xarray_time_dim)
        print(kwargs_with_xarray_time_dim)
        
        all_dims_in_args_and_kwargs = []
        for arg in args:
            try:
                all_dims_in_args_and_kwargs.extend(arg.dims)
            except AttributeError:
                pass
        for kwarg_value in kwargs.values():
            try:
                all_dims_in_args_and_kwargs.extend(kwarg_value.dims)
            except AttributeError:
                pass
            
        print(all_dims_in_args_and_kwargs)
        dims_to_loop_over = set([dim for dim in all_dims_in_args_and_kwargs if dim != 'time'])
        print(dims_to_loop_over)
        
        if len(dims_to_loop_over) > 1:
            raise NotImplementedError('Looping over more than one dim is not yet implemented.')
            
        data_list = []
        
        # iterate over the other dims
            
    return inner

test_func = decorator(pycml.processing.baseline.baseline_constant)
test_func(trsl=cml.rsl, wet=cml.rsl > 0);

[]
[{'trsl': <xarray.DataArray 'rsl' (channel_id: 2, time: 15840)>
array([[-42. , -42. , -42. , ..., -42. , -42. , -42. ],
       [-41.3, -41. , -41. , ..., -41. , -41. , -41. ]], dtype=float32)
Coordinates:
  * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
    cml_id            <U17 'SY8534_2_SY2370_2'
    length            float64 ...
    site_a_latitude   float64 ...
    site_a_longitude  float64 ...
    site_b_latitude   float64 ...
    site_b_longitude  float64 ...
  * channel_id        (channel_id) object 'channel_1' 'channel_2'
    frequency         (channel_id) float64 ...
    polarization      (channel_id) object ...}, {'wet': <xarray.DataArray 'rsl' (channel_id: 2, time: 15840)>
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
  * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
    cml_id            <U17 'SY8534_2_SY2370_2'
    length           

In [13]:
'time' in cml.rsl.dims

True

In [51]:
cml['wet'] = cml.trsl.rolling(time=60, center=True).std(skipna=False) > 0.8

cml['wet_fraction'] = (cml.wet==1).sum() / (cml.wet==0).sum()

cml['baseline'] = pycml.processing.baseline.baseline_constant(
    trsl=cml.trsl, 
    wet=cml.wet, 
    n_average_last_dry=5,
)
cml['waa_schleiss'] = pycml.processing.wet_antenna.waa_schleiss_2013(
    rsl=cml.trsl, 
    baseline=cml.baseline, 
    wet=cml.wet, 
    waa_max=2.2, 
    delta_t=1, 
    tau=15,
)

cml['A'] = cml.trsl - cml.baseline
cml['A'] = cml.A.where((cml.A.isnull().values | (cml.A.values >= 0)), 0)

cml['waa_leijnse'] = pycml.processing.wet_antenna.waa_leijnse_2008_from_A_obs(
    A_obs=cml.A,
    f_Hz=cml.frequency,
    L_km=cml.length,
    T_K=293.0,
    gamma=2.06e-05,
    delta=0.24,
    n_antenna=(1.73+0.014j),
    l_antenna=0.001,
)

TypeError: only size-1 arrays can be converted to Python scalars

In [29]:
fig, ax = plt.subplots()
cml.waa_leijnse.plot.line(x='time', ax=ax)
cml.waa_schleiss.plot.line(x='time', ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
cml

<xarray.Dataset>
Dimensions:           (channel_id: 2, time: 15840)
Coordinates:
  * time              (time) datetime64[ns] 2018-05-10 ... 2018-05-20T23:59:00
    cml_id            <U17 'SY8534_2_SY2370_2'
    length            float64 ...
    site_a_latitude   float64 ...
    site_a_longitude  float64 ...
    site_b_latitude   float64 ...
    site_b_longitude  float64 ...
  * channel_id        (channel_id) object 'channel_1' 'channel_2'
    frequency         (channel_id) float64 2.497e+10 2.598e+10
    polarization      (channel_id) object ...
Data variables:
    rsl               (channel_id, time) float32 -42.0 -42.0 ... -41.0 -41.0
    tsl               (channel_id, time) float32 16.0 16.0 16.0 ... 16.0 16.0
    trsl              (channel_id, time) float32 58.0 58.0 58.0 ... 57.0 57.0
    wet               (channel_id, time) bool False False False ... False False
    wet_fraction      float64 0.06223
    baseline          (channel_id, time) float64 58.0 58.0 58.0 ... 57.0 57.0
    waa               (channel_id, time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    A                 (channel_id, time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    R                 (channel_id, time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [7]:
import pycomlink as pycml
import numpy as np
import matplotlib.pyplot as plt

In [9]:
cml = pycml.io.examples.get_one_cml()

100%|██████████| 1/1 [00:00<00:00, 34.51it/s]


In [11]:
fig, ax = plt.subplots()

cml['tsl'] = cml.tsl.where(cml.tsl != 255.0)
cml['rsl'] = cml.rsl.where(cml.rsl != -99.9)

cml['trsl'] = cml.tsl - cml.rsl
cml.trsl.plot.line(x='time');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
cml['wet'] = cml.trsl.rolling(time=60, center=True).std() > 0.8

# Testing STFT method 

In [15]:
import matplotlib.mlab as mlab
import scipy.signal as signal
import numpy.testing

In [16]:
spec, f, t = mlab.specgram(cml.isel(channel_id=0).trsl.values, Fs=1/60, NFFT=64, window=np.hamming(64), noverlap=63)
f_sp, t_sp, spec_sp = signal.spectrogram(cml.isel(channel_id=0).trsl.values, fs=1/60, nfft=64, window=np.hamming(64), noverlap=63, detrend=None)

In [17]:
numpy.testing.assert_almost_equal(f, f_sp)

In [18]:
numpy.testing.assert_almost_equal(t, t_sp)

In [19]:
numpy.testing.assert_almost_equal(spec, spec_sp)

In [20]:
cml.isel(channel_id=0).trsl.values.shape

(41181,)

In [21]:
spec.shape

(33, 41118)

In [36]:
pycml.processing.wet_dry.stft.stft_classification(cml.isel(channel_id=0).trsl.values, window_length=64, threshold=1, f_divide=1e-3, dry_length=600)

AttributeError: module 'matplotlib.mlab' has no attribute 'find'

In [9]:
import matplotlib.pyplot as plt